In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

In [2]:
def get_clf_eval(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred, average='micro')
    precision = precision_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
    print('Accuracy:{0:.4f}, Recall:{1:.4f}, Precision:{2:.4f}, F1-Score:{3:.4f}'.format(accuracy, recall, precision, f1))

In [3]:
def make_confusion(y_test, y_pred):
    confusion = confusion_matrix(y_test,y_pred)
    confusion_df = pd.DataFrame(confusion,columns=['Predicted_Negative','Predicted_Positive'],index=['Actual_Negative','Actual_Positive'])
    
    return confusion_df

In [4]:
def load_dataset(directory):

    global input_cnt, output_cnt, data, x, y

    data = pd.read_csv(directory)
    input_cnt = data.loc[:, data.columns != 'Class Label'].shape[1]
    # output_cnt = data['Class Label'].shape[1]
    output_cnt = 1
    y = data['Class Label'].to_numpy()
    x = data.loc[:, data.columns != 'Class Label'].to_numpy()

In [31]:
class DiscriminentAnalysis():
    def __init__(self, alpha=0.0, beta=0.0):
        self.learned = False
        self.alpha = alpha
        self.beta = beta
        self.class_names = []
        self.class_priors = {}
        self.class_means = {}
        self.regularized_covariances = {}
        self.rda_covariances = {}
        self.reset()

    def reset(self):
        self.learned = False
        self.class_names = []
        self.class_priors = {}
        self.class_means = {}
        self.regularized_covariances = {}
        self.rda_covariances = {}

    def fit(self, X, y):
        self.class_names = np.unique(y)
        class_covariances = {}
        pooled_covariances = 0
        for i in self.class_names:
            class_indices = np.where(y == i)[0]
            class_samples = X[class_indices, :]
            self.class_priors[i] = float(len(class_indices)) / len(y)
            self.class_means[i] = np.mean(class_samples, axis=0)
            class_covariances[i] = np.cov(class_samples, rowvar=0)
            pooled_covariances += class_covariances[i] * self.class_priors[i]
        # Calculate RDA regularized covariance matricies for each class
        for i in self.class_names:
            self.regularized_covariances[i] = (self.beta * pooled_covariances) + ((1 - self.beta) *class_covariances[i])
            # self.regularized_covariances[i] = (self.beta * class_covariances[i]) + ((1 - self.beta) * pooled_covariances)

        for i in self.class_names:
            self.rda_covariances[i] = ((1-self.alpha) * self.regularized_covariances[i]) + (self.alpha * (1/self.class_priors[i]) * np.trace(self.regularized_covariances[i]) * np.eye(self.regularized_covariances[i].shape[0]))
        
        self.learned = True
        return self

    def predict(self, x):
        if not self.learned:
            raise NameError('Fit model first')
        # Determine probability of each class given input vector
        
        class_prob = {}
        for i in self.class_names:
            # Divid the class delta calculation into 3 parts
            part1 = -0.5 * np.log1p(np.linalg.det(self.rda_covariances[i]))
            # part2 = -0.5 * np.dot(np.dot((x - self.class_means[i]).T, np.linalg.pinv(self.rda_covariances[i])), (x - self.class_means[i]))
            part2 = -0.5 * np.matmul(np.matmul((x - self.class_means[i]).T, np.linalg.pinv(self.rda_covariances[i])), (x - self.class_means[i]))
            part3 = np.log(self.class_priors[i])
            class_prob[i] = part1 + part2 + part3
        return max(class_prob, key=class_prob.get)

In [66]:
class GridSearchRDA():
    def __init__(self, model, param_grid):
        self.model = model
        self.param_grid = param_grid
        self.alpha = 0
        self.beta = 0
        self.best_covariance = {}
        self.best_score = 0


    def fit(self, X, y, cv=3):
        
        # metric_score = []

        alpha_list = self.param_grid['alpha']
        beta_list = self.param_grid['beta']

        cv_x = np.split(X, cv)
        cv_y = np.split(y, cv)

        for alpha in alpha_list:
            for beta in beta_list:
                score_list = []
                for i in range(cv):
                    self.model.reset()
                    self.model.alpha = alpha
                    self.model.beta = beta

                    test_x_cv = cv_x[i]
                    train_x_cv = np.vstack(cv_x[:i] + cv_x[i + 1:])

                    test_y_cv = cv_y[i]
                    train_y_cv = np.vstack(cv_y[:i] + cv_y[i + 1:]).flatten()

                    
                    self.model.fit(train_x_cv, train_y_cv)

                    pred = []

                    for data in test_x_cv:
                        pred.append(self.model.predict(data))
                    
                    score_list.append(f1_score(test_y_cv, pred, average='micro'))
                
                mean_score = np.mean(np.array(score_list))
                print("alpha:{0:.1f}, beta:{1:.1f}, f1-score:{2:.8f}".format(alpha, beta,mean_score))

                if mean_score > self.best_score:
                    self.best_score = mean_score
                    self.alpha = alpha
                    self.beta = beta
                    self.best_covariance = self.model.rda_covariances
                    # self.best_estimator = self.model

In [67]:
load_dataset('./facial_expression_train_dataset.csv')

In [68]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=104)

In [69]:
y_train

array([1, 2, 2, 1, 2, 2, 3, 1, 3, 1, 1, 3, 2, 3, 3, 2, 3, 2, 1, 2, 1, 2,
       3, 3, 2, 1, 2, 1, 3, 2, 1, 2, 3, 1, 2, 1, 3, 2, 3, 2, 1, 3, 1, 1,
       3, 3, 3, 2, 3, 1, 1, 1, 2, 2, 3, 2, 3, 1, 3, 1, 2, 1, 3, 1, 1, 3,
       2, 3, 2], dtype=int64)

In [70]:
dis = DiscriminentAnalysis()

In [71]:
params = {
    'alpha': np.linspace(0.0, 1.0, num=11, endpoint=True),
    'beta': np.linspace(0.0, 1.0, num=11, endpoint=True)
}

In [72]:
grid = GridSearchRDA(dis, params)

In [73]:
grid.fit(x_train, y_train, cv=3)

alpha:0.0, beta:0.0, f1-score:0.82608696
alpha:0.0, beta:0.1, f1-score:0.97101449
alpha:0.0, beta:0.2, f1-score:0.95652174
alpha:0.0, beta:0.3, f1-score:0.95652174
alpha:0.0, beta:0.4, f1-score:0.95652174
alpha:0.0, beta:0.5, f1-score:0.95652174
alpha:0.0, beta:0.6, f1-score:0.95652174
alpha:0.0, beta:0.7, f1-score:0.97101449
alpha:0.0, beta:0.8, f1-score:0.95652174
alpha:0.0, beta:0.9, f1-score:0.95652174
alpha:0.0, beta:1.0, f1-score:0.95652174
alpha:0.1, beta:0.0, f1-score:0.60869565
alpha:0.1, beta:0.1, f1-score:0.62318841
alpha:0.1, beta:0.2, f1-score:0.66666667
alpha:0.1, beta:0.3, f1-score:0.66666667
alpha:0.1, beta:0.4, f1-score:0.65217391
alpha:0.1, beta:0.5, f1-score:0.62318841
alpha:0.1, beta:0.6, f1-score:0.53623188
alpha:0.1, beta:0.7, f1-score:0.47826087
alpha:0.1, beta:0.8, f1-score:0.52173913
alpha:0.1, beta:0.9, f1-score:0.52173913
alpha:0.1, beta:1.0, f1-score:0.60869565
alpha:0.2, beta:0.0, f1-score:0.42028986
alpha:0.2, beta:0.1, f1-score:0.42028986
alpha:0.2, beta:

In [74]:
grid.alpha

0.0

In [75]:
grid.beta

0.1

In [76]:
grid.best_score

0.9710144927536232

In [77]:
grid.best_estimator.beta

1.0

In [79]:
prediction = []
for testing in x_test:
    prediction.append(grid.best_estimator.predict(testing))

In [80]:
prediction

[2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2]

In [81]:
f1_score(y_test, prediction, average='micro')

0.5

In [55]:
y_test[9]

3

In [ ]:
rda_cov = {int(k):v.tolist() for k,v in cov.items()}

In [ ]:
# import json
# with open('covariance.json','w') as f:
#     json.dump(rda_cov, f)